In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN_co.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 5203.85it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       
        sort_order = sort_cryst_like_BGW(
            cryst = gkspc.g_cryst, key_array = gkspc.gk_norm2  
        )

        print(f"indices_gspace_sorted {indices_gspace_sorted}")
        print(f"sort_order {sort_order}")

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv1 = reorder_2d_matrix_sorted_gvecs(epsinv0, sort_order)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))
        std_eps2 = np.std(epsref - epsinv1) / np.sqrt(np.prod(list(epsinv0.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

        if np.abs(std_eps2) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps2}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 1/1 [00:00<00:00, 304.55it/s]

indices_gspace_sorted [ 0 14  2 11  5  3  7  1 10 12  6 13  9  4  8]
sort_order [ 0 14  2 11  5  3  7  1 10 12  6 13  9  4  8]
Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0
Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0


In [6]:
# Construct the Charge Matrix Class
from mtxel import ChargeMtxEL

cmtxclass = ChargeMtxEL.from_BGW(
    wfndata = wfndata,
    epsinp = epsinp,
    sigmainp = sigmainp,
    num_bands_val = 1,
    num_bands_con = 1,
)

In [7]:
mvc = cmtxclass.mvc(0)
print(f"Shape of mvc is {mvc.shape}")

mccp = cmtxclass.mccp(0, 0)
print(f"Shape of mccp is {mccp.shape}")

mvvp = cmtxclass.mvvp(0, 0)
print(f"Shape of mvvp is {mvvp.shape}")

Shape of mvc is (1, 1, 27)
Shape of mccp is (1, 1, 27)
Shape of mvvp is (1, 1, 27)


In [8]:
from read_mtxel import *
bgw_arrays = parse_file(dirname + "kernel.out")

mccp_BGW = bgw_arrays.get('mccp', None)
mvvp_BGW = bgw_arrays.get('mvvp', None)
mvc_BGW = bgw_arrays.get('mvc', None)
mvpcp_BGW = bgw_arrays.get('mvpcp', None)

In [9]:
print(f"Shape of mccp_BGW is {mccp_BGW.shape}")
print(f"Shape of mvvp_BGW is {mvvp_BGW.shape}")
print(f"Shape of mvc_BGW is {mvc_BGW.shape}")
print(f"Shape of mvpcp_BGW is {mvpcp_BGW.shape}")

Shape of mccp_BGW is (27, 1, 1, 1)
Shape of mvvp_BGW is (27, 1, 1, 1)
Shape of mvc_BGW is (27, 1, 1, 1)
Shape of mvpcp_BGW is (27, 1, 1, 1)


In [10]:
is_equal_mvc_mvpcp = np.allclose(mvc_BGW, mvpcp_BGW)
print(f"Is mvc and mvpcp equal? {is_equal_mvc_mvpcp}")

Is mvc and mvpcp equal? True


In [11]:
mccp = mccp[0, 0, :]
mccp_BGW = mccp_BGW[:, 0, 0, 0]

# for i in range(27):
#     mccp_value = mccp[i] if i < len(mccp) else 0
#     print(f"mccp[{i}]: {mccp_value.real:.10f}{mccp_value.imag:+.10f}j, mccp_BGW[{i}]: {mccp_BGW[i].real:.10f}{mccp_BGW[i].imag:+.10f}j")

are_mccp_equal = np.allclose(mccp, mccp_BGW)
print(f"Are mccp and mccp_BGW equal? {are_mccp_equal}")

Are mccp and mccp_BGW equal? True


In [12]:
mvc = mvc[0, 0, :]
mvc_BGW = mvc_BGW[:, 0, 0, 0]

# for i in range(27):
#     mvc_value = mvc[i] if i < len(mvc) else 0
#     print(f"mvc[{i}]: {mvc_value.real:.10f}{mvc_value.imag:+.10f}j, mvc_BGW[{i}]: {mvc_BGW[i].real:.10f}{mvc_BGW[i].imag:+.10f}j")

are_mvc_equal = np.allclose(mvc, mvc_BGW)
print(f"Are mvc and mvc_BGW equal? {are_mvc_equal}")

Are mvc and mvc_BGW equal? True


In [13]:
mvvp = mvvp[0, 0, :]
mvvp_BGW = mvvp_BGW[:, 0, 0, 0]

# for i in range(27):
#     mvvp_value = mvvp[i] if i < len(mvvp) else 0
#     print(f"mvvp[{i}]: {mvvp_value.real:.4f}{mvvp_value.imag:+.4f}j, mvvp_BGW[{i}]: {mvvp_BGW[i].real:.4f}{mvvp_BGW[i].imag:+.4f}j")

are_mvvp_equal = np.allclose(mvvp, mvvp_BGW)
print(f"Are mvvp and mvvp_BGW equal? {are_mvvp_equal}")

Are mvvp and mvvp_BGW equal? True


In [14]:
from read_vcoul import *
vcoul = load_v_coul(dirname + "vcoul_kernel")
vcoul = vcoul[0]


In [15]:
print(f"vcoul_kernel: {vcoul}")


vcoul_kernel: [ 0.        22.077974  22.077974  22.077974  22.077974  22.077974
 22.077974  22.077974  22.077974  16.55848   16.55848   16.55848
 16.55848   16.55848   16.55848    8.2792401  8.2792401  8.2792401
  8.2792401  8.2792401  8.2792401  8.2792401  8.2792401  8.2792401
  8.2792401  8.2792401  8.2792401]


In [16]:
volume = cmtxclass.crystal.reallat.cellvol
print(f"Volume: {volume}")

Volume: 265.3019999999999


In [17]:
sum = 0

for i in range(27):
    sum = sum + mvc[i] * vcoul[i] * np.conj(mvc[i])

print(f"Sum: {sum}")

Sum: (9.883808598983572+5.549824390497955e-17j)
